# Detroit Blight Ticket Compliance

According to [this CNBC article](https://www.cnbc.com/2014/05/28/eliminating-blight-could-cost-bankrupt-detroit-more-than-850-million.html), eliminating blight in the city of Detroit is estimated to cost \$850 million. Fines are issued to infractors, but many remain unpaid. Motivated by a [competition](https://www.kaggle.com/competitions/detroit-blight-ticket-compliance/overview) that took place six ago, this study tackles blight ticket compliance.

We will read, format and analyse two data sets containing training and test data seperately. The first will be used to fit various machine learning models, evaluated using a [AUC-ROC Curve](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) criteria. The latter will be used to predict if blight ticket will be paid on time.

As there are two categories for our prediction target - will be paid on time or it won't: this is a classification problem.

<div style="width:100%;text-align: center;"> <img align=middle src="https://image.cnbcfm.com/api/v1/image/101425482-DetroitWinter8.jpg?v=1394564303&w=630&h=354" alt="Heat beating" style="height:300px;margin-top:1rem;margin-bottom:1rem;"> </div>

## 1.    Data preprocessing

### 1.1 Importing libraries



In [1]:
# Data wrangling
import numpy as np 
import pandas as pd

# Data visualization
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns

### 1.2    Data reading

The main data sets considered in this study are:
* **training_data** (from *train.csv*): training set, containing tickets issued from 2004 to 2011;
* **test_data** (from *test.csv*): test set, containing tickets issued from 2012 to 2016.

Two complementary data sets that combined provides geographic information:
* **adresses** (from *addresses.csv*): mapping from ticket id to addresses;
* **coordinates** (from *latlons.csv*): mapping from addresses to latitude and longitude coordinates.

In [2]:
# Reading training set
training_data = pd.read_csv('/kaggle/input/detroit-blight-ticket/train.csv',
                           index_col='ticket_id',                           
                           encoding="ISO-8859-1",
                           low_memory=False)
training_data.name = 'training_data'

# Reading test set
test_data = pd.read_csv('/kaggle/input/detroit-blight-ticket/test.csv',
                       index_col='ticket_id')
test_data.name = 'test_data'

# Reading addresses
addresses = pd.read_csv('/kaggle/input/detroit-blight-ticket/addresses.csv')
addresses.name = 'addresses'

# Reading coordinates
coordinates = pd.read_csv('/kaggle/input/detroit-blight-ticket/latlons.csv')
coordinates.name = 'coordinates'

In [3]:
def columns(df):
    """From a dataframe df, get a report of column names"""
    
    columns = ', '.join(df.columns)
    length = len(df.columns)
    plural = 's' if length-1 else ''
    
    print(f"DataFrame '{df.name}' has {length} column{plural}: \n"+ columns+'.\n')
    
for df in [training_data, test_data, addresses, coordinates]:
    columns(df)

DataFrame 'training_data' has 33 columns: 
agency_name, inspector_name, violator_name, violation_street_number, violation_street_name, violation_zip_code, mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country, ticket_issued_date, hearing_date, violation_code, violation_description, disposition, fine_amount, admin_fee, state_fee, late_fee, discount_amount, clean_up_cost, judgment_amount, payment_amount, balance_due, payment_date, payment_status, collection_status, grafitti_status, compliance_detail, compliance.

DataFrame 'test_data' has 26 columns: 
agency_name, inspector_name, violator_name, violation_street_number, violation_street_name, violation_zip_code, mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country, ticket_issued_date, hearing_date, violation_code, violation_description, disposition, fine_amount, admin_fee, state_fee, late_fee, discount_amount, clean_up_cost, judgment_amount, g

### 1.3 Combining DataFrames

Before starting our analysis, *addresses* and *coordinates* combined will map latitude and longitude from ticket IDs for both *training_data* and *test_data*.

In [4]:
# Combining 'addresses' and 'coordinates' to create a 'mapping' DataFrame
mapping = addresses.merge(coordinates,
                         how='left',
                         on='address',
                         validate='m:1')
mapping.set_index('ticket_id',
                 inplace=True)

# Checking how many addresses didn't get coordinates mapped
mapping[mapping.lat.isnull() | mapping.lon.isnull()]

,address,lat,lon
ticket_id,,,
89535,"20424 bramford, Detroit MI",NaN,NaN
223598,"445 fordyce, Detroit MI",NaN,NaN
280256,"8300 fordyce, Detroit MI",NaN,NaN
317124,"20424 bramford, Detroit MI",NaN,NaN
329689,"8325 joy rd, Detroit MI 482O4",NaN,NaN
329393,"1201 elijah mccoy dr, Detroit MI 48208",NaN,NaN
333990,"12038 prairie, Detroit MI 482O4",NaN,NaN
367165,"6200 16th st, Detroit MI 482O8",NaN,NaN


In [5]:
# There are seven addresses that didn't get coordinates mapped, so we will correct it manually
addresses_and_coordinates = {'20424 bramford, Detroit MI':[42.446541, -83.023300],
                             '8300 fordyce, Detroit MI':[42.383251, -83.058189],
                             '445 fordyce, Detroit MI':[42.328590, -83.051460],
                             '8325 joy rd, Detroit MI 482O4':[42.358910, -83.151329],
                             '1201 elijah mccoy dr, Detroit MI 48208':[42.35891, -83.08291],
                             '12038 prairie, Detroit MI 482O4':[42.37675, -83.14319],
                             '6200 16th st, Detroit MI 482O8':[42.35995, -83.09583]}

for adr, crd in addresses_and_coordinates.items():
    mapping.loc[mapping.address==adr, ['lat', 'lon']] = crd

# Check if all entries are non-null
mapping.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311307 entries, 22056 to 369851
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   address  311307 non-null  object 
 1   lat      311307 non-null  float64
 2   lon      311307 non-null  float64
dtypes: float64(2), object(1)
memory usage: 9.5+ MB


In [6]:
# Combining 'mapping' with both 'training_data' and 'test_data'
training_mapped_data = training_data.join(mapping, how='left')
training_mapped_data.name = 'training_mapped_data'

test_mapped_data = test_data.join(mapping, how='left')
test_mapped_data.name = 'test_mapped_data'

### 1.4 Primary feature selection

Not all columns available in both *test_data* and *training_data* will be considered features. Discarding columns that will not be considered as features beforehand will reduce data cleaning complexity. Some primary features will need further feature engineerting before proceding to evaluation and machine learning.

The following table illustrates the process of choosing primary features.

| Column                      | Dtype   | Both | Description                                         | Feature | Reason |
| :-------------------------- | :-----: | :--: | :---------------------------------------------------| :-----: | :-- |
| agency_name                 | object  | ✔️   | Agency that issued the ticket                       | ✅     | 
| inspector_name              | object  | ✔️   | Name of inspector that issued the ticket            | 🔲     | Only 26% of inspectors appear in both test and training sets
| violator_name               | object  | ✔️   | Name of the infractor that the ticket was issued to | 🔲     | Only 2.8% of violators appear in both test and training sets
| violation_street_number     | float64 | ✔️   | Address where the violation occurred                | 🔲     | Mapped by latitude and longitude
| violation_street_name       | object  | ✔️   | Street where the violation occurred                 | 🔲     | Mapped by latitude and longitude
| violation_zip_code          | float64 | ✔️   | Zip code where the violation occurred               | 🔲     | Mapped by latitude and longitude
| mailing_address_str_number  | float64 | ✔️   | Mailing address of the violator                     | 🔲     | Mailing information not considered
| mailing_address_str_name    | object  | ✔️   | Mailing street of the violator                      | 🔲     | Mailing information not considered
| city                        | object  | ✔️   | Mailing city of the violator                        | 🔲     | Mailing information not considered
| state                       | object  | ✔️   | Mailing state of the violator                       | 🔲     | Mailing information not considered
| zip_code                    | object  | ✔️   | Mailing zip code of the violator                    | 🔲     | Mailing information not considered
| non_us_str_code             | object  | ✔️   | Mailing non us country code of the violator         | 🔲     | Mailing information not considered
| country                     | object  | ✔️   | Country of the violator                             | 🔲     | Mailing information not considered
| ticket_issued_date          | object  | ✔️   | Date and time the ticket was issued                 | 🔲     | This information alone is not descriptive
| hearing_date                | object  | ✔️   | Date and time the violator's hearing was scheduled  | 🔲     | A third is missing from test data
| violation_code              | object  | ✔️   | Type of violation                                   | ✅     | Some filtering is needed to combine code classes
| violation_description       | object  | ✔️   | Violation description                               | 🔲     | Target information encoded in *violation_code*
| disposition                 | object  | ✔️   | Judgment and judgement type                         | ✅     |
| fine_amount                 | float64 | ✔️   | Violation fine amount, excluding fees               | 🔲     | Part of *judment_amount*
| admin_fee                   | float64 | ✔️   | \$20 fee assigned to responsible judgments          | 🔲     | Constant to all violators deemed responsible
| state_fee                   | float64 | ✔️   | \$10 fee assigned to responsible judgments          | 🔲     | Constant to all violators deemed responsible
| late_fee                    | float64 | ✔️   | 10% fee assigned to responsible judgments           | ✅     | Combined with *discount_amount* into a new feature
| discount_amount             | float64 | ✔️   | 10% discount applied                                | ✅     | Combined with *late_fee* into a new feature
| clean_up_cost               | float64 | ✔️   | DPW clean-up or graffiti removal cost               | 🔲     | Constant zero to all training data
| judgment_amount             | float64 | ✔️   | Sum of all fines and fees                           | ✅     |
| payment_amount              | float64 | ❌   | Amount paid, if any                                 | 🔲     | Information not available at test-time
| balance_due                 | float64 | ❌   | Fines and fees still owed                           | 🔲     | Information not available at test-time
| payment_date                | object  | ❌   | Date payment was made, if it was received           | 🔲     | Information not available at test-time
| payment_status              | object  | ❌   | Current payment status as of February 1st 2017      | 🔲     | Irrelevant for new tickets
| collection_status           | object  | ❌   | Flag for payments in collections                    | 🔲     | Information not available at test-time
| grafitti_status             | object  | ✔️   | Flag for graffiti violations                        | 🔲     | Appears only once in training data
| compliance_detail           | object  | ❌   | Information on why it was/wasn't marked compliant   | 🔲     | Target information encoded in *compliance*
| compliance                  | float64 | ❌   | **Target variable for prediction**                  | ☑️     |
| address                     | object  | ✔️   | Full address where the violation occurred           | 🔲     | Mapped by latitude and longitude
| lat                         | float64 | ✔️   | Latitude where the violation occurred               | ✅     |
| lon                         | float64 | ✔️   | Longitude where the violation occurred              | ✅     |

Our target variable for prediction *compliance* has three states in *training_mapped_data*:
* **Null:** Not responsible;
* **0:** Responsible and non-compliant;
* **1:** Responsible and compliant

In order to analyse compliance, only violators deemed responsible are going to be considered.

In [7]:
# Selecting primary features before any feature engineering
primary_features = ['violation_code', 'agency_name', 'disposition', 'judgment_amount', 'late_fee', 'discount_amount', 'lat', 'lon']
target = ['compliance']

# Removing instances deemed not responsible
training_mapped_data.dropna(subset=['compliance'], inplace=True) 

# Checking type and if there are null primary feature values for both datasets
for df in [training_mapped_data, test_mapped_data]:
    print(f"Information about '{df.name}' DataFrame:")
    df[primary_features].info()
    print('\n')

# Visualizing primary features and target in training data
training_mapped_data.loc[:,primary_features+target]

Information about 'training_mapped_data' DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 22056 to 284333
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   violation_code   159880 non-null  object 
 1   agency_name      159880 non-null  object 
 2   disposition      159880 non-null  object 
 3   judgment_amount  159880 non-null  float64
 4   late_fee         159880 non-null  float64
 5   discount_amount  159880 non-null  float64
 6   lat              159880 non-null  float64
 7   lon              159880 non-null  float64
dtypes: float64(5), object(3)
memory usage: 11.0+ MB


Information about 'test_mapped_data' DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   violation_code   61001 non-null  object 
 1   agen

,violation_code,agency_name,disposition,judgment_amount,late_fee,discount_amount,lat,lon,compliance
ticket_id,,,,,,,,,
22056,9-1-36(a),"Buildings, Safety Engineering & Env Department",Responsible by Default,305.0,25.0,0.0,42.390729,-83.124268,0.0
27586,61-63.0600,"Buildings, Safety Engineering & Env Department",Responsible by Determination,855.0,75.0,0.0,42.326937,-83.135118,1.0
22046,9-1-36(a),"Buildings, Safety Engineering & Env Department",Responsible by Default,305.0,25.0,0.0,42.145257,-83.208233,0.0
18738,61-63.0500,"Buildings, Safety Engineering & Env Department",Responsible by Default,855.0,75.0,0.0,42.433466,-83.023493,0.0
18735,61-63.0100,"Buildings, Safety Engineering & Env Department",Responsible by Default,140.0,10.0,0.0,42.388641,-83.037858,0.0
...,...,...,...,...,...,...,...,...,...
285106,22-2-88(b),Department of Public Works,Responsible by Default,250.0,20.0,0.0,42.440228,-83.154829,0.0
284650,22-2-88(b),Department of Public Works,Responsible by Default,1130.0,100.0,0.0,42.406293,-83.171850,0.0
285125,22-2-88(b),Department of Public Works,Responsible by Default,580.0,50.0,0.0,42.366529,-83.141897,0.0


Thus, from the selected primary features three are categorical and five are numerical.

### 1.5 Feature engineering

#### 1.5.1 Violation code group

There are too many violation codes due to being specified down to articles for each violation code group.
We are interested in observing a broader effect of violations, therefore instances will be grouped.

Also, there are *violation_code* instances observed in *test_mapped_data* that are not present in *training_mapped_data*. Despite feature encoding dealing with this discrepancy, these instances will be renamed to *'Unknown'* in order to better illustrate data set contents during the exploratory data analysis section.


In [8]:
# List of 'violation_code' not seen during training phase, totalling of 94 instances.
unknown = '|'.join(['61-5-14', '61-85.0100', '19410901', '22-2-48', '19910901', '61-103.0100', '61-112.0100',
                    '61-14-175', '9-1-444', '61-13-102', '61-119.0100', '61-100.0100', '9-1-308', '9-1-471',
                    '9-1-218', '22-2-92', '61-5-20', '61-102.0100'])

suffixes = '|'.join(["[(].*", " .*", "[/].*"])

for df in [training_mapped_data, test_mapped_data]:
    df['violation_code_group'] = df.violation_code.str.replace(suffixes, "", regex=True)
    
test_mapped_data['violation_code_group'] = test_mapped_data.violation_code_group.str.replace(unknown, "Unknown", regex=True)

violation_code_group = pd.DataFrame([training_mapped_data.violation_code_group.value_counts(), test_mapped_data.violation_code_group.value_counts()]).T
violation_code_group.columns = ['training', 'test']
violation_code_group

,training,test
9-1-36,64441.0,8667.0
22-2-88,24187.0,17028.0
9-1-81,23160.0,1609.0
9-1-104,16927.0,16259.0
22-2-45,4200.0,2844.0
...,...,...
9-1-462,1.0,NaN
9-1-474,1.0,NaN
9-1-353,1.0,1.0
20160901,1.0,NaN


#### 1.5.2 Disposition

Similarly to violation code, similar dispositions will be grouped.

In [9]:
similar = {'Responsible (Fine Waived) by Deter':'Responsible by Determination',
           'Responsible - Compl/Adj by Default':'Responsible by Default',
           'Responsible - Compl/Adj by Determi':'Responsible by Determination',
           'Responsible (Fine Waived) by Admis':'Responsible by Admission',
           'Responsible by Dismissal':'Unknown'}

for df in [training_mapped_data, test_mapped_data]:
    df.disposition.replace(similar, inplace=True)

disposition = pd.DataFrame([training_mapped_data[training_mapped_data.compliance.notna()].disposition.value_counts(), test_mapped_data.disposition.value_counts()]).T
disposition.columns = ['training', 'test']
disposition

,training,test
Responsible by Default,138340.0,51608.0
Responsible by Admission,13701.0,4485.0
Responsible by Determination,7839.0,4907.0
Unknown,NaN,1.0


#### 1.5.3 Fee or discount

Both *late_fee* and *discount_amount* represent 10% of *judgment_amount*. They would apply a fee or a discount to judgment amount respectively and do not occur simutaneously in the same instance. Therefore we can reduce these two primary features into a new feature 'fee_or_discount': with three possible values:
* **-1:** discount was applied;
* **1:** late fee was applied;
* **0:** no modifier.

In [10]:
for df in [training_mapped_data, test_mapped_data]:
    df['fee_or_discount'] = np.sign(df.late_fee-df.discount_amount)

training_mapped_data.fee_or_discount.value_counts()

 1.0    144422
 0.0     14278
-1.0      1180
Name: fee_or_discount, dtype: int64

#### 1.5.4 Final features

After feature engineering, we come to the following group of features.

In [11]:
features = ['violation_code_group', 'agency_name', 'disposition', 'judgment_amount', 'fee_or_discount', 'lat', 'lon']

# Checking type and if there are null primary feature values for both datasets
for df in [training_mapped_data, test_mapped_data]:
    print(f"Information about '{df.name}' DataFrame:")
    df[features].info()
    print('\n')

# Visualizing missing features and target values in training data
training_mapped_data.loc[:,features+target]

Information about 'training_mapped_data' DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 22056 to 284333
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   violation_code_group  159880 non-null  object 
 1   agency_name           159880 non-null  object 
 2   disposition           159880 non-null  object 
 3   judgment_amount       159880 non-null  float64
 4   fee_or_discount       159880 non-null  float64
 5   lat                   159880 non-null  float64
 6   lon                   159880 non-null  float64
dtypes: float64(4), object(3)
memory usage: 9.8+ MB


Information about 'test_mapped_data' DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   violation_code_group  61001 non-null  objec

,violation_code_group,agency_name,disposition,judgment_amount,fee_or_discount,lat,lon,compliance
ticket_id,,,,,,,,
22056,9-1-36,"Buildings, Safety Engineering & Env Department",Responsible by Default,305.0,1.0,42.390729,-83.124268,0.0
27586,61-63.0600,"Buildings, Safety Engineering & Env Department",Responsible by Determination,855.0,1.0,42.326937,-83.135118,1.0
22046,9-1-36,"Buildings, Safety Engineering & Env Department",Responsible by Default,305.0,1.0,42.145257,-83.208233,0.0
18738,61-63.0500,"Buildings, Safety Engineering & Env Department",Responsible by Default,855.0,1.0,42.433466,-83.023493,0.0
18735,61-63.0100,"Buildings, Safety Engineering & Env Department",Responsible by Default,140.0,1.0,42.388641,-83.037858,0.0
...,...,...,...,...,...,...,...,...
285106,22-2-88,Department of Public Works,Responsible by Default,250.0,1.0,42.440228,-83.154829,0.0
284650,22-2-88,Department of Public Works,Responsible by Default,1130.0,1.0,42.406293,-83.171850,0.0
285125,22-2-88,Department of Public Works,Responsible by Default,580.0,1.0,42.366529,-83.141897,0.0


From the initial eight primary features, two were modified in place and other two were combined into one. From the new group of seven features, three are categorical and four are numerical. Also, there are no null values.

### 1.6 Exploratory data analisys (EDA)

In this section, first we analyse summary statistics for both categorical and numerical features. Then we will explore how these different features influence compliance.

In [12]:
# Separate features into categorical and numerical
features_categorical = ['violation_code_group', 'agency_name', 'disposition']
features_numerical = ['judgment_amount', 'fee_or_discount', 'lat', 'lon']

# Setting preferences for plots
sns.set_theme(style="white", palette="deep")

#### 1.6.1 Categorical features summary

The table below shows summary statistics for categorical features in both training and test sets. The number of different categories in *violation_code* is far greater than the other two categorical features. In general, test set show less unique classes than training sets, except for the *'Unknown'* classes introduced during feature engineering and will be dealt with during encoding phase. As noted before, there are no null values in each category.

Each categorical feature will be explored further on its respective section.

In [13]:
training_categorical_summary = training_mapped_data[features_categorical].describe()
test_categorical_summary = test_mapped_data[features_categorical].describe()

pd.concat([training_categorical_summary, test_categorical_summary], keys=['training','test'])

violation_code_group  \
training count                159880   
         unique                  146   
         top                  9-1-36   
         freq                  64441   
test     count                 61001   
         unique                   93   
         top                 22-2-88   
         freq                  17028   

                                                    agency_name  \
training count                                           159880   
         unique                                               5   
         top     Buildings, Safety Engineering & Env Department   
         freq                                             95863   
test     count                                            61001   
         unique                                               3   
         top                         Department of Public Works   
         freq                                             40731   

                            disposition  
training count                   159880  
         unique                       3  
         top     Responsible by Default  
         freq                    138340  
test     count                    61001  
         unique                       4  
         top     Responsible by Default  
         freq                     51608

#### 1.6.2 Numerical features summary

The table below shows summary statistics for numerical features in both training and test sets. 

*Compliance* is also included when summarizing training data, but it is absent from test data. Interestingly, only 7.3% of the tickets in training data have 'complied' status. This means our future machine learning models might be skewed towards prediciting non-compliance, thus making accuracy an insufficient metric.

Each numerical feature will be explored further on its respective section.

In [14]:
training_numerical_summary = training_mapped_data[features_numerical+target].describe()
test_numerical_summary = test_mapped_data[features_numerical].describe()

pd.concat([training_numerical_summary, test_numerical_summary], keys=['training','test'])

judgment_amount  fee_or_discount            lat  \
training count    159880.000000    159880.000000  159880.000000   
         mean        420.650218         0.895934      42.389746   
         std         742.555062         0.328630       0.037120   
         min           0.000000        -1.000000      42.145257   
         25%         250.000000         1.000000      42.364535   
         50%         305.000000         1.000000      42.392575   
         75%         305.000000         1.000000      42.419160   
         max       11030.000000         1.000000      44.785389   
test     count     61001.000000     61001.000000   61001.000000   
         mean        347.895541         0.855478      42.392625   
         std         460.058043         0.385514       0.037672   
         min           0.000000        -1.000000      42.230964   
         25%          85.000000         1.000000      42.365163   
         50%         250.000000         1.000000      42.402047   
         75%         305.000000         1.000000      42.422812   
         max       15558.800000         1.000000      42.554454   

                          lon     compliance  
training count  159880.000000  159880.000000  
         mean      -83.111529       0.072536  
         std         0.093515       0.259374  
         min       -84.403766       0.000000  
         25%       -83.185854       0.000000  
         50%       -83.124517       0.000000  
         75%       -83.033202       0.000000  
         max       -82.512936       1.000000  
test     count   61001.000000            NaN  
         mean      -83.117657            NaN  
         std         0.098964            NaN  
         min       -83.791406            NaN  
         25%       -83.201210            NaN  
         50%       -83.138942            NaN  
         75%       -83.038791            NaN  
         max       -82.888926            NaN

#### 1.6.1 Violation code group

In [15]:
# Get ticket count and compliance rate for training data 
violation_code_group = training_mapped_data[['violation_code_group', 'compliance']].groupby('violation_code_group').agg(['mean', 'count'])
violation_code_group = violation_code_group.droplevel(0, axis='columns')

# Get ticket count for test data
violation_code_group = violation_code_group.join(test_mapped_data.violation_code_group.value_counts()).fillna(0)

# Renaming columns and selecting dtype for plotting
violation_code_group.rename(columns={'mean': 'training_compliance_rate',
                                     'count': 'training_count',
                                     'violation_code_group': 'test_count'},
                                     inplace=True)
violation_code_group.test_count = violation_code_group.test_count.astype('int32')

# Sorting columns in descending training count
violation_code_group.sort_values('training_count', inplace=True, ascending=False)
violation_code_group

,training_compliance_rate,training_count,test_count
violation_code_group,,,
9-1-36,0.074037,64441,8667
22-2-88,0.069211,24187,17028
9-1-81,0.032081,23160,1609
9-1-104,0.082235,16927,16259
22-2-45,0.131905,4200,2844
...,...,...,...
9-1-354,0.000000,1,0
61-104.0100,0.000000,1,1
9-1-406,0.000000,1,0


In [16]:
# Ticket count and compliance rate by violation code group in training data
hexplot = sns.jointplot(data=violation_code_group,
                        x="training_count",
                        y="training_compliance_rate",
                        ratio=4,
                        height=7,
                        kind="hex",
                        xscale='log',
                        gridsize=30,
                        mincnt=1,
                        edgecolors='gray',
                        vmin=0,
                        vmax=20,
                        marginal_kws=dict(bins=20, fill=False))

# Make auxiliary axis to insert colorbar
colorbar_axis = inset_axes(hexplot.ax_joint,
                           width="30%",
                           height="2%",
                           loc="upper right")
plt.colorbar(cax=colorbar_axis, orientation="horizontal", label='Number of groups');

# Finishing
hexplot.set_axis_labels('Number of blight tickets (logarithmic scale)', 'Compliance rate', labelpad=7);
plt.suptitle('Training data by violation group code', y=.98);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Even after feature engineering, there are 146 violation code groups.
As the plot suggests, the majority of violation code groups contain only one ticket and have a compliance rate of 0%.
Between all violation code groups, the compliance rate is mainly lower than 20%.

#### 1.6.2 Agency name

In [17]:
# Get count and mean compliance for training data 
agency_name = training_mapped_data[['agency_name', 'compliance']].groupby('agency_name').agg(['mean', 'count'])
agency_name = agency_name.droplevel(0, axis='columns')

# Get count for test data
agency_name = agency_name.join(test_mapped_data.agency_name.value_counts().astype('int64')).fillna(0)

# Renaming columns and selecting dtype for plotting
agency_name.rename(columns={'mean': 'training_compliance_rate',
                            'count': 'training_count',
                            'agency_name': 'test_count'},
                   inplace=True)
agency_name.test_count = agency_name.test_count.astype('int32')

agency_name

,training_compliance_rate,training_count,test_count
agency_name,,,
"Buildings, Safety Engineering & Env Department",0.060743,95863,16832
Department of Public Works,0.089961,52445,40731
Detroit Police Department,0.131720,4464,3438
Health Department,0.065851,7107,0
Neighborhood City Halls,0.000000,1,0


All agencies in test data appear in training data. Only *'Health Department'* and *'Neighborhood City Halls'* appear in training data but not on test data. The latter appears only once in training data, and for this reason, we choose to omit it in the graphic analysis. These instances will not be excluded from the data set though, they might have relevant information regarding other features.

In [18]:
# Setting the plot figure
fig, (ax1, ax2) = plt.subplots(1, 2, 
                               figsize=(13,4),
                               gridspec_kw={'width_ratios': [1, 3]},
                               sharey=True)
fig.suptitle('Training data by agency name', y=0.9)

drop = agency_name.drop('Neighborhood City Halls') # Removing single instance from graph   

# Color preference
color = 'midnightblue'

# First plot
first_bars = ax1.barh(drop.index, 100*drop.training_compliance_rate,
                      facecolor=color)

ax1.barh(drop.index, 100*(1-drop.training_compliance_rate),
         left=100*drop.training_compliance_rate,
         color='silver')
ax1.bar_label(first_bars, padding=8, fmt='%.1f%%')
ax1.get_xaxis().set_visible(False)
ax1.set_xlim(0,100)
ax1.set_title('Compliance rate')

# Second plot
second_bars = ax2.barh(drop.index, drop.training_count,
                       facecolor=color)
ax2.invert_yaxis()
ax2.bar_label(second_bars, padding=8)
ax2.get_xaxis().set_visible(False)
ax2.set_title('Number of blight tickets')

# Finishing
sns.despine(left = True, bottom = True)
plt.tight_layout()
plt.subplots_adjust(top=.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 1.6.3 Disposition

In [19]:
# Get count and mean compliance for training data 
disposition = training_mapped_data[['disposition', 'compliance']].groupby('disposition').agg(['mean', 'count'])
disposition = disposition.droplevel(0, axis='columns')

# Get count for test data
disposition = disposition.join(test_mapped_data.disposition.value_counts()).fillna(0)

# Renaming columns for plotting
disposition.rename(columns={'mean': 'training_compliance_rate',
                            'count': 'training_count',
                            'disposition': 'test_count'},
                   inplace=True)
disposition.test_count = disposition.test_count.astype('int32')

# Sorting columns in descending training count
disposition.sort_values('training_count', inplace=True, ascending=False)
disposition

,training_compliance_rate,training_count,test_count
disposition,,,
Responsible by Default,0.038203,138340,51608
Responsible by Admission,0.275016,13701,4485
Responsible by Determination,0.324531,7839,4907


In [20]:
# Setting the plot figure
fig, (ax1, ax2) = plt.subplots(1, 2, 
                               figsize=(13,3),
                               gridspec_kw={'width_ratios': [1, 3]},
                               sharey=True)
fig.suptitle('Training data by disposition', y=0.9)

# First plot
first_bars = ax1.barh(disposition.index, 
                      100*disposition.training_compliance_rate,
                      facecolor=color)

ax1.barh(disposition.index, 
         100*(1-disposition.training_compliance_rate),
         left=100*disposition.training_compliance_rate,
         color='silver')
ax1.bar_label(first_bars, padding=8, fmt='%.1f%%')
ax1.get_xaxis().set_visible(False)
ax1.set_xlim(0,100)
ax1.set_title('Compliance rate')

# Second plot
second_bars = ax2.barh(disposition.index, 
                       disposition.training_count,
                       facecolor=color)
ax2.invert_yaxis()
ax2.bar_label(second_bars, padding=8)
ax2.get_xaxis().set_visible(False)
ax2.set_title('Number of blight tickets')

# Finishing
sns.despine(left = True, bottom = True)
plt.tight_layout()
plt.subplots_adjust(top=.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The far numerous disposition *'Responsible by Default'* has a much lower compliance percentage than the total average of 7.3. The other two dispositions have a much higher compliance percentage but are much less prevalent in training data. 


#### 1.6.4 Judgment amount

#### 1.6.5 Fee or discount

#### 1.6.6 Latitude and Longitude

In this section we will analyse how compliance appears on map for training instances. Non-compliant and compliant tickets will be plotted separately to see how they are spread on the map. There is a clear concentration of tickets near the metropolitan area, despite the data set showing instances located far into non-metropolitan area.

For each compliance status, both the metropolitan and the full area maps will be displayed.

In [21]:
offset = 0.1 # Latitude and longitude offset for plotting purposes
quantile = 0.01 # Percentage of 'outer' locations to be disregarded

# Area comprised of ticket locations
bounding_box = (training_mapped_data.lon.min()-offset, training_mapped_data.lon.max()+offset,
                training_mapped_data.lat.min()-offset, training_mapped_data.lat.max()+offset)

# Selecting locations between quantile and 1-quantile
mask = (training_mapped_data.lon > training_mapped_data.lon.quantile(quantile))&\
       (training_mapped_data.lon < training_mapped_data.lon.quantile(1-quantile))&\
       (training_mapped_data.lat > training_mapped_data.lat.quantile(quantile))&\
       (training_mapped_data.lat < training_mapped_data.lat.quantile(1-quantile))

training_zoomed_data = training_mapped_data[mask]

# Bounding box reduced by quantile margins
zoomed_box = (training_zoomed_data.lon.min()-offset, training_zoomed_data.lon.max()+offset,
              training_zoomed_data.lat.min()-offset, training_zoomed_data.lat.max()+offset)

print(f'Bounding box: {bounding_box}\nZoomed box:{zoomed_box}')

Bounding box: (-84.5037662, -82.41293610000001, 42.045257, 44.8853893)
Zoomed box:(-83.3727309, -82.8329533, 42.189582, 42.545895)


In [22]:
# Non compliant tickets will be mapped using a hexbin map
mask = training_mapped_data.compliance==0
non_compliant = training_mapped_data[mask]
compliant = training_mapped_data[~mask]

def hexbin_map(ax, image_path, box, data, gridsize, cmap):
    """Generates non-compliant tickets hexbin plots for both general and zoomed maps"""

    # Data can be either compliant or non-compliant
    hexbin = ax.hexbin(data.lon,
                       data.lat,
                       gridsize=gridsize,
                       mincnt=1,
                       alpha=0.5,
#                        edgecolor='black',
                       cmap=cmap)
    
    # Setting title and background map
    detroit_map = plt.imread('../input/detroit-blight-ticket/'+image_path)
    ax.imshow(detroit_map, zorder=0, extent = box, aspect= 'equal');

    # Setting
    ax.set_xlim(box[0],box[1])
    ax.set_ylim(box[2],box[3])
    
    # Hide axis ticks
    ax.set_xticks([])
    ax.set_yticks([])
    
    return hexbin
    
def inset_axis(ax, data, gridsize, cmap, title):
    """Nests zoomed map into full map"""
    
    # Plot main map
    main_map = hexbin_map(ax, 'All_tickets.png', bounding_box, data, gridsize, cmap)
    
    # Make auxiliary axis to insert colorbar
    colorbar_axis = inset_axes(ax,
                               width="30%",
                               height="2%",
                               loc="upper right")
    fig.colorbar(main_map,
                 cax=colorbar_axis,
                 orientation="horizontal",
                 label=title);
    
    # Create an inset axis in the bottom right corner
    ax_zoom = ax.inset_axes([0.20, 0.09, 1, 1])

    # Plot zoomed map
    hexbin_map(ax_zoom, 'zoomed_view.png', zoomed_box, data, gridsize, cmap)

    # Add the lines to indicate where the inset axis is coming from
    ax.indicate_inset_zoom(ax_zoom,
                           edgecolor='black',
                           alpha=1)

# Setting figure and plotting maps
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (13,7))
inset_axis(ax1, non_compliant, 200, 'autumn', 'Non-compliant')
inset_axis(ax2, compliant, 40, 'summer', 'Compliant')
fig.suptitle('Ticket compliance by coordinates')
plt.subplots_adjust(top=.88)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As previously noted, there are far more instances of non-compliant tickets. The highest non-compliant ticket concentration between the center and west. Another concentration can be seen northeast.

The concentration of compliant tickets roughly follows the non-compliant spread, but the highest concentration of compliant tickets can be observed south of the western non-compliant concentration.